In [1]:
import numpy as np 
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.core import Dense, Activation, Flatten, Dropout
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [2]:
cd E:\Caltech_Winter\CS 155\mini project1

E:\Caltech_Winter\CS 155\mini project1


In [3]:
from numpy import genfromtxt
import numpy as np
train_file = 'training_data.txt'
test_file = 'test_data.txt'
train_data = genfromtxt(train_file, delimiter=' ')
test_data = genfromtxt(test_file, delimiter=' ')
train_data = train_data[1:]

In [4]:
def NeuralRun(X_train, y_train, num_epoch, dropout, num_hidden_x, learning_rate):
    model = Sequential()
    model.add(Dense(num_hidden_x, input_shape=(len(X_train[0]),)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    # compile network
    Adam = keras.optimizers.Adam(lr = learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=Adam, metrics=['accuracy'])

    fit = model.fit(X_train, y_train, batch_size = 32, epochs=num_epoch, verbose=0)

    return model

In [5]:
def CrossValidation(train_data, num_epoch, fold, dropout, num_hidden_x, learning_rate):
    
    np.random.seed(123456)
    ind = np.random.permutation(train_data.shape[0])
    new_train_data = train_data[ind].astype(int)

    # Here, y = 0 means bad review, and y = 1 means good review
    X_train = new_train_data[:, 1:]
    y_train = new_train_data[:, 0]

    kf = KFold(n_splits=fold)
    max_acc = 0
    i = 1
    
    print('Parameters: [Epochs] %d, [Fold] %d; [Dropout] %s, [Hidden X] %d, [Learning Rate] %s' 
          % (num_epoch, fold, dropout, num_hidden_x, learning_rate))
    print('Start Training...')
    for train_index, test_index in kf.split(X_train):
        # print('Train on Fold %1d' % i)
        i = i + 1
        
        model = NeuralRun(X_train[train_index], y_train[train_index], num_epoch, dropout, num_hidden_x, learning_rate)
        
        # Printing the accuracy of our model, according to the loss function specified in model.compile
        score = model.evaluate(X_train[test_index], y_train[test_index], verbose=0)
        # print('Test score:', score[0])
        # print('Test accuracy:', score[1])
        
        if score[1] > max_acc:
            max_model = model
            max_acc = score[1]
    
    print('Best accuracy:', max_acc)
    return max_model

In [11]:
fold = 5
dropout = 0.2
num_hidden_x = 70
learning_rate = 0.002
num_epoch = 20

np.random.seed(123456)
ind = np.random.permutation(train_data.shape[0])
new_train_data = train_data[ind].astype(int)

# Here, y = 0 means bad review, and y = 1 means good review
X_train = new_train_data[:, 1:]
y_train = new_train_data[:, 0]
X_test = test_data[1:]

model = NeuralRun(X_train, y_train, num_epoch, dropout, num_hidden_x, learning_rate)
y_test = model.predict(X_test)

In [15]:
y_test_int = [1 if i > 0.5 else 0 for i in y_test]

with open("NNTest.txt", 'a') as text_file:
    text_file.write("Id,Prediction\n")
    for i in range(len(y_test_int)):
        text_file.write("{},{}\n".format(i+1, y_test_int[i]))

In [14]:
print(X_test)

[[2. 3. 1. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 1. ... 0. 0. 0.]
 [3. 1. 0. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]
